In [3]:
import json

import pandas as pd
import requests

from api_key import jeju_cits_api_key

In [2]:
park_url = f"http://api.jejuits.go.kr/api/infoParkingInfoList?code={jeju_cits_api_key}"
response = requests.get(park_url)

In [4]:
park_data = response.json()

In [13]:
park_data

{'result': 'success',
 'info_cnt': 10,
 'info': [{'id': '11111111',
   'name': '서귀포매일올레시장',
   'addr': '서귀포시 중앙로 62번길 18',
   'x_crdn': 126.56326295,
   'y_crdn': 33.25031562,
   'park_day': '월화수목금토일',
   'wkdy_strt': '090000',
   'wkdy_end': '180000',
   'lhdy_strt': '090000',
   'lhdy_end': '180000',
   'basic_time': 30,
   'basic_fare': 1000,
   'add_time': 15,
   'add_fare': 500,
   'whol_npls': 216},
  {'id': '16488201',
   'name': '법원북측공영주차장',
   'addr': '제주시 이도이동 1066',
   'x_crdn': 126.53534209,
   'y_crdn': 33.49472463,
   'park_day': '월화수목금토일',
   'wkdy_strt': '090000',
   'wkdy_end': '180000',
   'lhdy_strt': '090000',
   'lhdy_end': '180000',
   'basic_time': 30,
   'basic_fare': 1000,
   'add_time': 15,
   'add_fare': 500,
   'whol_npls': 91},
  {'id': '20019319',
   'name': '산짓물공영주차장',
   'addr': '제주시 건입동 1330',
   'x_crdn': 126.52872778,
   'y_crdn': 33.51590058,
   'park_day': '월화수목금토일',
   'wkdy_strt': '090000',
   'wkdy_end': '180000',
   'lhdy_strt': '090000',
   'lh

In [14]:
print(len(park_data['info']))

10


In [10]:
# Extract relevant information from the data dictionary
info_list = park_data['info']
filtered_data = [{'PARK_NM': item['name'], 'Latitude': item['y_crdn'], 'Longitude': item['x_crdn']} for item in info_list]
# Add '주차장' to PARK_NM if it does not end with '주차장'
for item in filtered_data:
    if '주차' not in item['PARK_NM']:
        item['PARK_NM'] += '주차장'

# Convert the filtered data to a pandas DataFrame
park_df = pd.DataFrame(filtered_data)
print(park_df.head(10))

        PARK_NM   Latitude   Longitude
0  서귀포매일올레시장주차장  33.250316  126.563263
1     법원북측공영주차장  33.494725  126.535342
2      산짓물공영주차장  33.515901  126.528728
3     이도2동공영주차장  33.496738  126.534944
4      북수구공영주차장  33.514692  126.527941
5      신제주공영주차장  33.488938  126.491068
6   동문공설시장공영주차장  33.512089  126.528200
7    서귀중앙공영주차빌딩  33.249857  126.562211
8      칠성골공영주차장  33.516579  126.525116
9       인제공영주차장  33.504793  126.541950


In [11]:
print(len(park_df))
park_df.to_csv("data/JEJU_PARKING.csv", encoding="cp949", index=False)

10


In [17]:
tour_url = f"http://api.jejuits.go.kr/api/infoTourList?code={jeju_cits_api_key}"
response = requests.get(tour_url)
tour_data = response.json()

In [18]:
print(len(tour_data['info']))

1136


In [21]:
print(tour_data['info'][:5])

[{'contents_id': 'CNTS_000000000022691', 'contents_label': '관광지', 'title': '오라동 청보리-메밀꽃밭', 'address': '제주특별자치도 제주시 오라2동 산 76', 'road_address': '제주특별자치도 제주시 오라2동 산 76', 'tag': '경관/포토,친구,커플,맑음,봄,가을', 'introduction': '한라산 아래 드넓은 청보리-메밀꽃의 향연', 'latitude': 33.426865, 'longitude': 126.505775, 'post_code': '', 'phone_no': '--', 'img_path': 'https://api.cdn.visitjeju.net/photomng/imgpath/201809/28/fbc56229-ade0-47d9-a6be-c148ee791dc6.JPG', 'thumbnail_path': 'https://api.cdn.visitjeju.net/photomng/thumbnailpath/201809/28/42c86682-e38b-4ceb-b429-89591ba870b1.JPG'}, {'contents_id': 'CONT_000000000500589', 'contents_label': '관광지', 'title': '조은승마장', 'address': '제주특별자치도 서귀포시 안덕면 동광리 897-2', 'road_address': '제주특별자치도 서귀포시 안덕면 동광로 265-14', 'tag': '승마,액티비티,커플,가을', 'introduction': '아이들과 함께하는 승마체험', 'latitude': 33.318214, 'longitude': 126.344894, 'post_code': '63523', 'phone_no': '064-792-0550', 'img_path': 'https://api.cdn.visitjeju.net/photomng/imgpath/201804/30/f1dd27f5-56b2-4db7-a391-e8fa2a81a120.jpg'

In [23]:
# Extract relevant information from the data dictionary
tour_list = tour_data['info']
filtered_data = []
for item in tour_list:
    try:
        filtered_data.append({
            'TOUR_NM': item['title'],
            'Latitude': item['latitude'],
            'Longitude': item['longitude']
        })
    except KeyError:
        continue

# Convert the filtered data to a pandas DataFrame
tour_df = pd.DataFrame(filtered_data)
print(len(tour_df))

1136


In [30]:
def split_tour_names(row):
    tour_nm = row['TOUR_NM']
    
    # Check if there are parentheses
    if '(' in tour_nm and ')' in tour_nm:
        # Extract the main part and the parts inside the parentheses
        main_name = tour_nm.split('(')[0].strip()  # Main name
        inside_parentheses = tour_nm.split('(')[1].replace(')', '').strip()  # Sub-names inside parentheses
        
        # Split the contents inside the parentheses by commas
        sub_names = [name.strip() for name in inside_parentheses.split(',')]
        
        # Create a list of dictionaries for each new row
        new_rows = []
        
        # Add the main name as a row
        new_row_main = row.copy()
        new_row_main['TOUR_NM'] = main_name
        new_rows.append(new_row_main)
        
        # Add each sub-name as a new row
        for sub_name in sub_names:
            new_row_sub = row.copy()
            new_row_sub['TOUR_NM'] = sub_name
            new_rows.append(new_row_sub)
        
        return new_rows
    else:
        # Return the row as is if no parentheses
        return [row]

# Apply the splitting function and flatten the list of lists
expanded_rows = []
for idx, row in tour_df.iterrows():
    expanded_rows.extend(split_tour_names(row))

# Convert the expanded list of rows back into a DataFrame
expanded_tour_df = pd.DataFrame(expanded_rows)

# Display the expanded DataFrame
print(expanded_tour_df)

           TOUR_NM   Latitude   Longitude
0     오라동 청보리-메밀꽃밭  33.426865  126.505775
1            조은승마장  33.318214  126.344894
2            번개레이싱  33.302971  126.299068
3              관덕정  33.513355  126.521450
4             윗세오름  33.370172  126.481970
...            ...        ...         ...
1131   타보카 월정리투명카약  33.563460  126.780250
1132      핀크스 골프클럽  33.315998  126.387643
1133         와흘본향당  33.502320  126.639070
1134         해적잠수함  33.308254  126.164700
1135    제주올레 1-1코스  33.505802  126.941780

[1338 rows x 3 columns]


In [31]:
expanded_tour_df.head(10)

,TOUR_NM,Latitude,Longitude
0,오라동 청보리-메밀꽃밭,33.426865,126.505775
1,조은승마장,33.318214,126.344894
2,번개레이싱,33.302971,126.299068
3,관덕정,33.513355,126.521450
4,윗세오름,33.370172,126.481970
4,웃세오름,33.370172,126.481970
5,9.81 파크,33.392887,126.358632
6,붉은오름 자연휴양림,33.395905,126.669785
7,용두암,33.514962,126.511687
8,지미봉,33.489535,126.877942


In [33]:
print(len(expanded_tour_df))
expanded_tour_df.to_csv("data/JEJU_TOUR.csv", encoding="cp949", index=False)

1338
